# IMPORTS #

In [ ]:
pip install supervision

In [1]:
import torch
import pandas as pd
import numpy as np
from ultralytics.utils.ops import xyxy2xywh
from ultralytics.utils.plotting import Annotator, colors
import cv2
from ultralytics import YOLO
from matplotlib import pyplot as plt
# no longer required
#import fiftyone as fo
#import fiftyone.zoo as foz
import supervision as sv

# Getting the OpenImagesV7 dataset ready for use #

We are going to use the fiftyone lib to get the only the penguin data. \
Then we are going to need to make sure the data is compatible and in the right format for the YOLOv8 model.

## Get only the penguin data from the OpenImagesV7 dataset ##

### Using the fiftyone - this requires the MongaDB so I am going to just work with the dataset directly ###

In [ ]:
# test
# to be removed
dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    max_samples=100,
    seed=51,
    shuffle=True,
)

session = fo.launch_app(dataset.view())
# fiftyone is not working working off the bat so I am going to try something else and come back to this


### working with the dataset and annotations directly ###

data is found here: https://storage.googleapis.com/openimages/web/download_v7.html \
\
The goal is to create a list with all the image ids that specify that a penguin has been detected\
The downloader.py function that is supplied by OIV7 (and is saved it the colab directory) can be called and will only download the images in that list\
We then need to check those images and adjust the relevant annotations to only include those ids and save them.\


In [2]:
# path to the parent folder of the raw data

ROOT_RAW_OI7_PATH = '/home/matthew/Desktop/Master_Dev/masters_penguin_pose_estimation/data/raw/OpenImagesV7'

#### Getting the data annotations in a dataframe and filtering to only penguins ####

In [89]:
# load csvs to dataframes to work with
raw_image_labels = pd.read_csv(f'{ROOT_RAW_OI7_PATH}/test-annotations-human-imagelabels-boxable.csv')
raw_bbox_labels = pd.read_csv(f'{ROOT_RAW_OI7_PATH}/test-annotations-bbox.csv')

In [90]:
raw_image_labels.head()

,ImageID,Source,LabelName,Confidence
0,000026e7ee790996,verification,/m/0cgh4,0
1,000026e7ee790996,verification,/m/04hgtk,0
2,000026e7ee790996,verification,/m/0d5gx,0
3,000026e7ee790996,verification,/m/07j7r,1
4,000026e7ee790996,verification,/m/04rky,0


In [91]:
raw_bbox_labels.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000026e7ee790996,xclick,/m/07j7r,1,0.071875,0.145313,0.206250,0.391667,0,1,1,0,0
1,000026e7ee790996,xclick,/m/07j7r,1,0.439063,0.571875,0.264583,0.435417,0,1,1,0,0
2,000026e7ee790996,xclick,/m/07j7r,1,0.668750,1.000000,0.000000,0.552083,0,1,1,0,0
3,000062a39995e348,xclick,/m/015p6,1,0.206208,0.849224,0.154639,1.000000,0,0,0,0,0
4,000062a39995e348,xclick,/m/05s2s,1,0.137472,0.376940,0.000000,0.883652,1,1,0,0,0


the penguin id (found from the oldv7-class-descriptions-boxable.csv file):\
/m/05z6w

In [6]:
# below you can also just make it image labels where LabelName is the same but the confidence score is also equal to 1
# then you dont need to do the removing the rows thing

In [92]:
# create datasets that only contain entries with images in them
penguin_image_labels = raw_image_labels[raw_image_labels['LabelName'] == '/m/05z6w']
penguin_bbox_labels = raw_bbox_labels[raw_bbox_labels['LabelName'] == '/m/05z6w']

In [93]:
penguin_image_labels.head(97)

,ImageID,Source,LabelName,Confidence
6084,01e8e6894bf88777,verification,/m/05z6w,1
7654,025dfa9ea18bcb18,verification,/m/05z6w,0
14090,045f2a7a01277369,verification,/m/05z6w,0
17172,0555911ee31d049f,verification,/m/05z6w,0
24612,07ca230691eec10e,verification,/m/05z6w,1
...,...,...,...,...
482309,9ee32a851eba0eea,verification,/m/05z6w,1
492643,a25a74a471183cee,verification,/m/05z6w,1
495405,a32f24bc0220911f,verification,/m/05z6w,0
516197,a9ddfcc0e100968c,verification,/m/05z6w,1


In [94]:
# get number of rows (number of images)
penguin_image_labels.shape

(144, 4)

In [95]:
penguin_bbox_labels.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
7934,01e8e6894bf88777,xclick,/m/05z6w,1,0.050186,0.899628,0.014035,0.843860,0,0,0,0,0
30971,07ca230691eec10e,xclick,/m/05z6w,1,0.000000,0.466814,0.625369,1.000000,1,1,0,0,0
30972,07ca230691eec10e,xclick,/m/05z6w,1,0.327434,0.761062,0.122419,0.501475,1,0,0,0,0
30973,07ca230691eec10e,xclick,/m/05z6w,1,0.396018,0.818584,0.215339,0.971976,0,0,0,0,0
37464,099862b891b82c9a,xclick,/m/05z6w,1,0.526549,0.621681,0.309735,0.401180,0,0,0,1,0


In [96]:
# get number of rows (number of bboxs)
penguin_bbox_labels.shape

(165, 13)

In [97]:
# First I would like to just check that all the ids appear in both lists (sanity check)
# I will remove anything that does not appear in both 
def check_id_presence_in_dfs(df1, df2, id_column, df1_name='df1', df2_name='df2'):
    """
    Check and print IDs that are not present in both dataframes and put them in a list. prints and returns nothing if all ids are present.
    """
    # Set of ids from each df
    ids_df1 = set(df1[id_column])
    ids_df2 = set(df2[id_column])

    # create lists for ids to be removefd
    df1_list_2remove = []
    df2_list_2remove = []

    # Find IDs that are only in df1
    only_in_df1 = ids_df1 - ids_df2
    if only_in_df1:
        #print(f"IDs only in {df1_name}:")
        for id_ in only_in_df1:
            #print(id_)
            df1_list_2remove.append(id_)

    # Find IDs that are only in df2
    only_in_df2 = ids_df2 - ids_df1
    if only_in_df2:
        #print(f"IDs only in {df2_name}:")
        for id_ in only_in_df2:
            #print(id_)
            df2_list_2remove.append(id_)

    return df1_list_2remove, df2_list_2remove

In [98]:
# if returns nothing then all images are present in both dfs
penguin_image_labels_2remove, penguin_bbox_labels_2remove = check_id_presence_in_dfs(penguin_image_labels, penguin_bbox_labels, 'ImageID', 'label', 'bbox')

In [99]:
print(len(penguin_image_labels_2remove))
print(len(penguin_bbox_labels_2remove))

47
0


In [100]:
# function to remove ids that don't appear in both lists
def remove_entries_by_ids(df, id_list, id_column):
    '''Removes rows from the DataFrame where the ID is in the provided list.'''
    filtered_df = df[~df[id_column].isin(id_list)]
    return filtered_df

In [101]:
penguin_image_labels = remove_entries_by_ids(penguin_image_labels, penguin_image_labels_2remove, 'ImageID')
penguin_bbox_labels = remove_entries_by_ids(penguin_bbox_labels, penguin_bbox_labels_2remove, 'ImageID')

In [64]:
penguin_image_labels.shape

(42, 4)

In [65]:
penguin_bbox_labels.shape

(88, 13)

#### creating the text file with a list of all the images the contain penguins and getting all the images from that and downloading those images to a folder - you only need to do this once for the test, train and val datasets on the ####

In [66]:
# create a text file with the following layout:
#$SPLIT/$IMAGE_ID, where $SPLIT is either "train", "test", "validation", or "challenge2018"; and $IMAGE_ID is the image ID that uniquely identifies the image
# so the text file will look like this:
#train/f9e0434389a1d4dd
#train/1a007563ebc18664
#test/ea8bfd4e765304db
# the following function should create the required text file:
def write_ids_to_file(location, name, df):
    """
    Writes entries to a text file where each entry is 'name/id' from the DataFrame's ID column.

    :param location: String, the directory where the text file will be saved.
    :param name: String, the name to be prefixed before each ID.
    :param df: DataFrame, the dataframe containing an 'id' column.
    """
    # Open a file at the specified location to write
    with open(f"{location}/{name}_ids.txt", "w") as file:
        # Loop through each id in the DataFrame
        for id_ in df['ImageID']:
            # Write formatted string to file
            file.write(f"{name}/{id_}\n")

In [102]:
# call the above function for the test images
# call the above function for the test images
# call the above function for the test images
write_ids_to_file(ROOT_RAW_OI7_PATH, 'test', penguin_image_labels)

Now run the following command in the terminal:\
python3 /home/matthew/Desktop/Master_Dev/masters_penguin_pose_estimation/notebooks/OpenImagesV7_downloader.py /home/matthew/Desktop/Master_Dev/masters_penguin_pose_estimation/data/raw/OpenImagesV7/image_list_file.txt --download_folder=/home/matthew/Desktop/Master_Dev/masters_penguin_pose_estimation/data/raw/OpenImagesV7/OpenImageV7_raw_images --num_processes=5

but replace image_list_file.txt with the name of the test file you just created.\
\
The images should be copied to the /home/matthew/Desktop/Master_Dev/masters_penguin_pose_estimation/data/raw/OpenImagesV7/OpenImageV7_raw_images directory 

In [21]:
# this will allow you to see the row corresponding to the id provided
input_id = '0d6a527e509d7e6e'
matching_row = penguin_image_labels[penguin_image_labels['ImageID'] == input_id]
print(matching_row)

Empty DataFrame
Columns: [ImageID, Source, LabelName, Confidence]
Index: []


In [22]:
# this will allow you to see the row corresponding to the id provided
input_id = '0d6a527e509d7e6e'
matching_row = penguin_bbox_labels[penguin_bbox_labels['ImageID'] == input_id]
print(matching_row)

Empty DataFrame
Columns: [ImageID, Source, LabelName, Confidence, XMin, XMax, YMin, YMax, IsOccluded, IsTruncated, IsGroupOf, IsDepiction, IsInside, XClick1X, XClick2X, XClick3X, XClick4X, XClick1Y, XClick2Y, XClick3Y, XClick4Y]
Index: []

[0 rows x 21 columns]


#### Create the annotation files ####
here we need to create a text file for each image with the following format for each bounding box in that image, where each bounding box is on a new line:\
0 x_centre_value y_centre_value width height \
\
note that all values are normalised between zero and 1

##### convert the bboxs to Yolo format: XCentre YCentre Width Height and save these as new columns to the df and save that dataframe to a .json file - might need to convert it to a series first

In [24]:
# first we need to create a function to get bounding boxes in an array in a format:
# x1 y1 x2 y2
# since the origin is in the top left corner, that is to say it must be:
# XMin Ymin XMax YMax
# test
def extract_values_to_array(df, column_names):
    """
    Extracts values from specified columns in a DataFrame and returns them as a numpy array.
    
    :param df: pandas DataFrame from which to extract values.
    :param column_names: List of column names whose values are to be extracted.
    :return: A numpy array with each row corresponding to the values from the specified columns in the DataFrame.
    """ 
    # Extract the values as a numpy array
    array_values = df[column_names].to_numpy()
    
    return array_values


In [69]:
# use the above function to obtain the desired array
list_col_to_extract = ['XMin', 'YMin', 'XMax', 'YMax']

penguin_bbox_labels_arr = extract_values_to_array(penguin_bbox_labels,list_col_to_extract)

print(penguin_bbox_labels_arr[0:10])

[[          0    0.037611      0.9174           1]
 [  0.0088496    0.027982     0.77876           1]
 [    0.12708    0.071875     0.86875      0.9375]
 [    0.11406       0.125           1     0.70417]
 [    0.31061     0.11532      0.9072      0.9191]
 [     0.1418     0.40618     0.26145     0.84106]
 [      0.226     0.38411     0.34121     0.81457]
 [    0.41654     0.46358     0.54653     0.85872]
 [    0.56573     0.50993     0.70162     0.88079]
 [          0     0.10417     0.19531     0.97292]]


In [70]:
print(penguin_bbox_labels.iloc[1])
# checking the output of the arrayh and it looks good

ImageID        0d2d4e6cefec6f87
Source                   xclick
LabelName              /m/05z6w
Confidence                    1
XMin                    0.00885
XMax                   0.778761
YMin                   0.027982
YMax                        1.0
IsOccluded                    0
IsTruncated                   0
IsGroupOf                     0
IsDepiction                   0
IsInside                      0
Name: 15896, dtype: object


In [71]:
# using the function from Yolo to convert from xyxy to xywh
penguin_bbox_labels_arr_xywh = xyxy2xywh(penguin_bbox_labels_arr)

In [72]:
print(penguin_bbox_labels_arr_xywh[0:10])

[[     0.4587     0.51881      0.9174     0.96239]
 [    0.39381     0.51399     0.76991     0.97202]
 [    0.49792     0.50469     0.74167     0.86562]
 [    0.55703     0.41458     0.88594     0.57917]
 [     0.6089     0.51721     0.59659     0.80379]
 [    0.20162     0.62362     0.11965     0.43488]
 [     0.2836     0.59934     0.11521     0.43046]
 [    0.48154     0.66115     0.12999     0.39514]
 [    0.63368     0.69536     0.13589     0.37086]
 [   0.097656     0.53854     0.19531     0.86875]]


In [73]:
# check the first entry is correct
# check centre x 
print(((penguin_bbox_labels_arr[0, 2] - penguin_bbox_labels_arr[0, 0])/2)+penguin_bbox_labels_arr[0, 0])
# check centre y 
print(((penguin_bbox_labels_arr[0, 3] - penguin_bbox_labels_arr[0, 1])/2)+penguin_bbox_labels_arr[0, 1])
# check centre w 
print((penguin_bbox_labels_arr[0, 2] - penguin_bbox_labels_arr[0, 0]))
# check centre h
print((penguin_bbox_labels_arr[0, 3] - penguin_bbox_labels_arr[0, 1]))

0.45870205
0.51880531
0.9174041
0.96238938


In [74]:
# convert arr to df and specify the column names
new_cols = ['XCentre', 'YCentre', 'Width', 'Height']
penguin_bbox_labels_arr_xywh_df = pd.DataFrame(penguin_bbox_labels_arr_xywh, columns=new_cols, index=penguin_bbox_labels.index)

In [75]:
penguin_bbox_labels_arr_xywh_df.head()

,XCentre,YCentre,Width,Height
3242,0.458702,0.518805,0.917404,0.962389
15896,0.393805,0.513991,0.769912,0.972018
26473,0.497917,0.504687,0.741667,0.865625
35255,0.557031,0.414583,0.885938,0.579167
35636,0.608902,0.517212,0.596591,0.803787


In [76]:
penguin_bbox_labels.shape

(88, 13)

In [77]:
penguin_bbox_labels_arr_xywh_df.shape

(88, 4)

In [78]:
# concat new df to the end of the original df
penguin_bbox_labels_xywh = pd.concat([penguin_bbox_labels, penguin_bbox_labels_arr_xywh_df], axis=1)

In [79]:
penguin_bbox_labels_xywh.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,XCentre,YCentre,Width,Height
3242,028d762d6716abdf,xclick,/m/05z6w,1,0.000000,0.917404,0.037611,1.000000,0,1,0,0,0,0.458702,0.518805,0.917404,0.962389
15896,0d2d4e6cefec6f87,xclick,/m/05z6w,1,0.008850,0.778761,0.027982,1.000000,0,0,0,0,0,0.393805,0.513991,0.769912,0.972018
26473,1654d220ef8dfb40,xclick,/m/05z6w,1,0.127083,0.868750,0.071875,0.937500,0,0,0,0,0,0.497917,0.504687,0.741667,0.865625
35255,1d9ddcae0b59b04c,xclick,/m/05z6w,1,0.114062,1.000000,0.125000,0.704167,0,0,1,0,0,0.557031,0.414583,0.885938,0.579167
35636,1e20429aac140db6,xclick,/m/05z6w,1,0.310606,0.907197,0.115318,0.919105,0,0,0,0,0,0.608902,0.517212,0.596591,0.803787


In [80]:
penguin_bbox_labels_xywh.shape

(88, 17)

In [81]:
# now save it as a .json file so we dont have to do this again
# we will use the orient = 'records' as this is the default and
# the most common
json_path_parent = '/home/matthew/Desktop/Master_Dev/masters_penguin_pose_estimation/data/raw/OpenImagesV7'
json_path = f'{json_path_parent}/validation_full_annotation.json'

penguin_bbox_labels_xywh.to_json(json_path, orient='records')

##### Run the below to load the full annotations to a df #####

In [ ]:
# the following code will read a .json file to a df
# just uncomment and run the code below changing the name of the json file
# json_path = ''/home/matthew/Desktop/Master_Dev/masters_penguin_pose_estimation/data/raw/OpenImagesV7/test_full_annotation.json'
# df = pd.read_json_into_df(json_path, orient='records')

##### step through the df and create a text file for each unique instance of an image id then add: 0 XCentre YCentre Width Height as a new line for each bounding box for a specific image id #####

In [82]:
# create a function 
def df_to_YOLO_txt_file(df, txt_file_path_parent):
    # step through the df
    for index, entry in df.iterrows():
        # get info from entry
        ImageID = entry['ImageID']
        XCentre = entry['XCentre']
        YCentre = entry['YCentre']
        Width = entry['Width']
        Height = entry['Height']

        txt_file_path = f'{txt_file_path_parent}/{ImageID}.txt'

        # write info to text file in desired format
        with open(txt_file_path, 'a') as file:
            file.write(f'0 {XCentre} {YCentre} {Width} {Height}\n')


In [83]:
# before running the below you need to check the number of files
# in the destination folder so you can see that the correct number of 
# txt files have been made (should = number of penguin_image_label)
penguin_image_labels.shape

(42, 4)

In [84]:
txt_file_path_parent = '/home/matthew/Desktop/Master_Dev/masters_penguin_pose_estimation/data/raw/OpenImagesV7/OI7_Yolo_annotations'
df_to_YOLO_txt_file(penguin_bbox_labels_xywh, txt_file_path_parent)

In [ ]:
# now check the number of files in the folder (just use file explorer)

#### View the results to check they are correct ####

##### attempt to use supervision #####

In [91]:
image = cv.imread('/home/matthew/Desktop/Master_Dev/masters_penguin_pose_estimation/data/raw/OpenImagesV7/OpenImageV7_raw_images/3df173193a2bac8b.jpg')
ann = Annotator(image)
txt_path = '/home/matthew/Desktop/Master_Dev/masters_penguin_pose_estimation/data/raw/OpenImagesV7/OI7_Yolo_annotations/3df173193a2bac8b.txt'
bbox_arr = np.loadtxt(txt_path, dtype=np.float16, delimiter=' ', usecols=(1,2,3,4))
bbox_tensor = torch.from_numpy(bbox_arr)
bbox_dict = {'predictions': tuple(row[:]) for row in bbox_arr}

In [92]:
print(bbox_arr)
print(bbox_tensor)
print(bbox_dict)

[[    0.40332     0.75977    0.051636     0.16589]
 [    0.51172     0.79297    0.097351     0.16589]
 [    0.58545     0.77637    0.061951     0.15491]]
tensor([[0.4033, 0.7598, 0.0516, 0.1659],
        [0.5117, 0.7930, 0.0974, 0.1659],
        [0.5854, 0.7764, 0.0620, 0.1549]], dtype=torch.float16)
{'predictions': (0.5854, 0.7764, 0.06195, 0.1549)}


In [93]:
detect1 = sv.Detections.from_inference(bbox_dict)

KeyError: 'image'

In [82]:
box_annotator = sv.BoundingBoxAnnotator()
annotated_frame = box_annotator.annotate(image.copy(), detections=bbox_tensor)
sv.plot_image(image=annotated_frame, size=(16, 16))

AttributeError: 'Tensor' object has no attribute 'xyxy'

##### Attempt to use cv #####

In [85]:
# display images inline
%matplotlib inline

In [103]:
image_path = '/home/matthew/Desktop/Master_Dev/masters_penguin_pose_estimation/data/raw/OpenImagesV7/OpenImageV7_raw_images/381820d77d917c77.jpg'
txt_path = '/home/matthew/Desktop/Master_Dev/masters_penguin_pose_estimation/data/raw/OpenImagesV7/OI7_Yolo_annotations/381820d77d917c77.txt'
bbox_arr = np.loadtxt(txt_path, dtype=np.float16, delimiter=' ', usecols=(1,2,3,4))

In [104]:
print(bbox_arr)

[    0.50195     0.55469      0.6665     0.89062]


In [105]:
def draw_bboxes_and_show(image_path, bboxes):
    """
    Draws bounding boxes on an image and displays it inline in a Jupyter Notebook.

    :param image_path: Path to the image file.
    :param bboxes: NumPy array of bbox in the format [x_center, y_center, width, height] as normalized values.
    """
    # Load the image
    img = cv2.imread(image_path)
    
    # Ensure the image was loaded
    if img is None:
        print("Image not loaded; please check the path.")
        return
    
    # Convert bounding box coordinates from normalized to absolute pixel values
    height, width = img.shape[:2]
    #print(height, width)
    scaled_bboxes = np.zeros_like(bboxes) # creating emptry array of correct size
    #print(scaled_bboxes)
    scaled_bboxes[:, 0] = (bboxes[:, 0] - (bboxes[:, 2] / 2))* width   # x_min
    scaled_bboxes[:, 1] = (bboxes[:, 1] - bboxes[:, 3] / 2)* height  # y_min
    scaled_bboxes[:, 2] = (bboxes[:, 0] + bboxes[:, 2] / 2)* width   # x_max
    scaled_bboxes[:, 3] = (bboxes[:, 1] + bboxes[:, 3] / 2)* height  # y_max
    
    #print(scaled_bboxes)
    # Draw each bounding box on the image
    for (x_min, y_min, x_max, y_max) in scaled_bboxes:
        cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (255, 0, 0), 2)
    
    # Convert BGR image to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Display the image
    plt.figure(figsize=(10, 10))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

In [106]:
#print(bbox_arr)
draw_bboxes_and_show(image_path, bbox_arr)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed